<a href="https://colab.research.google.com/github/asoane34/TF_object_detection/blob/master/03FasterRCNNInceptionResnetV2Atrous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount("/gdrive")

In [0]:
%cd /gdrive/'My Drive'/object_detection

/gdrive/My Drive/object_detection


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf

tf.__version__

'1.15.2'

In [0]:
def validate_dataset(filenames, reader_opts=None):

    i = 0
    
    for fname in filenames:
        
        print('validating ', fname)

        record_iterator = tf.io.tf_record_iterator(path=fname, options=reader_opts)
        
        try:
            
            for _ in record_iterator:
                
                i += 1
                
        except Exception as e:
            
            print('error in {} at record {}'.format(fname, i))
            
            print(e)

In [0]:
validate_dataset(["./global-wheat-detection/validation.tfrecord", "./global-wheat-detection/train.tfrecord",
                  "./global-wheat-detection/test_images.tfrecord"])

validating  ./global-wheat-detection/validation.tfrecord
validating  ./global-wheat-detection/train.tfrecord
validating  ./global-wheat-detection/test_images.tfrecord


In [0]:
%cd models/research 

!protoc object_detection/protos/*.proto --python_out=.

/gdrive/My Drive/object_detection/models/research


In [0]:
import os

os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':/gdrive/My Drive/object_detection/models/research/slim:/gdrive/My Drive/object_detection/models/research'

os.environ['PYTHONPATH']

'/tensorflow-1.15.2/python3.6:/env/python:/gdrive/My Drive/object_detection/models/research/slim:/gdrive/My Drive/object_detection/models/research'

In [0]:
!python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

# Model Selection

When selecting models, the key decision to make is latency versus accuracy. In this case, accuracy is more important than latency (I can't imagine why anyone would need to identify wheat heads quickly). The first model I trained was a __Faster R-CNN ResNet101 Model__, and the accuracy on the test set was still poor. I'm going to take a couple of steps to address this: 

* We are moving up the chain: Heading up the ladder to the __Faster R-CNN with Inception ResNet__. The training time is going to increase and we are going to lose latency for inferencing but who cares, our wheat heads aren't going anywhere. 

* Adding more data augmentation - The model did not translate well to the test set. After inspecting the images, it's not hard to see why. These wheat heads oftentimes don't like anything alike: They're different shapes, they're different colors, and sometimes they are extremely difficult to pick out of the background. I also noticed that the model was having trouble with images that contained larger wheat-heads, but it was better able to identify the smaller ones. The last model only employed Random Horizontal Flip for data augmentation - looking back that was a mistake. Horizontally flipping really makes no difference. This time I will be employing:
  * Random Image Cropping: This hopefully help the model pick wheat heads out of the background as well as identify the larger wheat heads. 
  * Random RGB to Gray: This will hopefully help the model identify shape better. 

In [0]:
%cd object_detection/

!wget -O model_training.tar.gz http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.tar.gz -q

!tar xvzf model_training.tar.gz

!rm model_training.tar.gz

/gdrive/My Drive/object_detection/models/research/object_detection
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/checkpoint
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/pipeline.config
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt.meta
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/saved_model/
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/saved_model/variables/
faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/frozen_inference_graph.pb


If we don't remove the checkpoint file from this directory, it will skip the model training because it already thinks it has gone enough steps. 

In [0]:
!rm faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/checkpoint

In [0]:
os.environ['DATA_PATH'] = '/gdrive/My Drive/object_detection/global-wheat-detection'

os.environ['MODEL_PATH'] = 'object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

It's important to set up export and Servo directories as well, the training API will throw an error at the end of training if you do not.

In [0]:
%cd object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/

!mkdir export

%cd export

!mkdir Servo

%cd ..

/gdrive/My Drive/object_detection/models/research/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28


In [0]:
%%writefile global_wheat_detection.config
model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_resnet_v2'
      first_stage_features_stride: 8
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 8
        width_stride: 8
      }
    }
    first_stage_atrous_rate: 2
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 17
    maxpool_kernel_size: 1
    maxpool_stride: 1
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 116
        max_total_detections: 116
      }
      score_converter: SIGMOID
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/gdrive/My Drive/object_detection/models/research/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  data_augmentation_options {
    random_rgb_to_gray { 
    }
    random_crop_image {
        random_coef: 0.5  
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/object_detection/global-wheat-detection/train.tfrecord"
  }
  label_map_path: "/gdrive/My Drive/object_detection/global-wheat-detection/label_map.pbtxt"
}

eval_config: {
  metrics_set: "pascal_voc_detection_metrics"
  use_moving_averages: false
  num_examples: 338
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/object_detection/global-wheat-detection/validation.tfrecord"
  }
  label_map_path: "/gdrive/My Drive/object_detection/global-wheat-detection/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Writing global_wheat_detection.config


In [0]:
cd ../..

tf.test.is_gpu_available()

/gdrive/My Drive/object_detection/models/research


True

In [0]:
!python object_detection/model_main.py \
    --pipeline_config_path=object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/global_wheat_detection.config \
    --model_dir=object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28 \
    --num_train_steps=40000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr=False

In [0]:
import numpy as np
import re

ckpts = [f for f in os.listdir('object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28') \
       if 'model.ckpt-' in f and '.meta' in f]

ckpt_steps = np.array([int(re.findall('\d+', f)[0]) for f in ckpts])

last_model = ckpts[ckpt_steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28', last_model)

In [0]:
print(last_model_path)

object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt-7902


In [0]:
output_dir = "object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/trained_model7902/"

if not os.path.exists(output_dir):

    os.makedirs(output_dir)

In [0]:
!python object_detection/export_inference_graph.py \
   --input_type=image_tensor \
   --pipeline_config_path object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/global_wheat_detection.config \
   --output_directory object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/trained_model7902/ \
   --trained_checkpoint_prefix object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt-7902

In [0]:
!python object_detection/inference/infer_detections.py \
  --input_tfrecord_paths=/gdrive/'My Drive'/object_detection/global-wheat-detection/test_images.tfrecord \
  --output_tfrecord_path=/gdrive/'My Drive'/object_detection/global-wheat-detection/model2_inferences7902.tfrecord \
  --inference_graph=object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/trained_model7902/frozen_inference_graph.pb \
  --discard_image_pixels